In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 


## Importing Data

In [38]:
df = pd.read_excel("da.xlsx")
df.head()

,Unnamed: 0,Date,ID_User,Type,Amount,Status,Volume
0,0,2021-01-01,1,Recharge,1.000000,1,100.000000
1,1,2021-01-01,2,Recharge,1.000000,1,100.000000
2,2,2021-01-01,7,Recharge,1.143444,1,114.344384
3,3,2021-01-01,10,Recharge,1.169300,1,116.929957
4,4,2021-01-01,11,Recharge,1.994728,1,199.472785


## Inspecting and Validation of Data 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   Unnamed: 0  1048575 non-null  int64         
 1   Date        1048575 non-null  datetime64[ns]
 2   ID_User     1048575 non-null  int64         
 3   Type        1048575 non-null  object        
 4   Amount      1048575 non-null  float64       
 5   Status      1048575 non-null  int64         
 6   Volume      1048575 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 56.0+ MB


In [40]:
# useless Column unnamed :
df.drop(columns= ["Unnamed: 0"], inplace= True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype         
---  ------   --------------    -----         
 0   Date     1048575 non-null  datetime64[ns]
 1   ID_User  1048575 non-null  int64         
 2   Type     1048575 non-null  object        
 3   Amount   1048575 non-null  float64       
 4   Status   1048575 non-null  int64         
 5   Volume   1048575 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 48.0+ MB


In [7]:
df.head()

,Date,ID_User,Type,Amount,Status,Volume
0,2021-01-01,1,Recharge,1.000000,1,100.000000
1,2021-01-01,2,Recharge,1.000000,1,100.000000
2,2021-01-01,7,Recharge,1.143444,1,114.344384
3,2021-01-01,10,Recharge,1.169300,1,116.929957
4,2021-01-01,11,Recharge,1.994728,1,199.472785


#### Checking missing Values 

In [8]:
df.isna().sum()

Date       0
ID_User    0
Type       0
Amount     0
Status     0
Volume     0
dtype: int64

##### No missing values but some attributes especially String attributes (attribute: Type) may interpret for example space string (" ") as non null, let's validate that all the entries are valid (Type in [Recharge, RTP])

In [9]:
df["Type"].unique()

array(['Recharge', 'RTP'], dtype=object)

In [10]:
df["Status"].unique()

array([1, 0], dtype=int64)

##### No missing values founded

#### Checking Duplicates : 
##### Removing duplicates is a must in the data quality checking process, **But** It is completely the opposit in our case, removing duplicates in this case will cause information loss, that is because, for example, a client could recharge his account multiple times (multiple operations) in the same day with the same ammount, resulting several records with the same data, _thus we should not remove the duplicated rows_

#### Luckily, The Data is clean and ready for analysis 

## Data Analysis :

#### Daily Number of Operation : Recharge and RTP 

In [11]:
daily_ops = df.groupby(["Date", "Type"]).agg("Volume").count()
daily_ops

Date        Type    
2021-01-01  Recharge    3836
2021-01-02  Recharge    3849
2021-01-03  Recharge    4027
2021-01-04  Recharge    3939
2021-01-05  Recharge    3741
                        ... 
2021-05-13  Recharge    3856
2021-05-14  RTP         3798
            Recharge    4020
2021-05-15  RTP         1072
            Recharge     821
Name: Volume, Length: 265, dtype: int64

### Monthly recharge and RTP operations

In [12]:
monthly_volume= df.groupby([df.Date.dt.month,"Type"]).agg("Volume").sum()/100
monthly_volume

Date  Type    
1     RTP        -435255.049280
      Recharge    714357.017969
2     RTP        -464362.840937
      Recharge    655456.955929
3     RTP        -510517.134215
      Recharge    720676.294169
4     RTP        -502619.735388
      Recharge    691237.961167
5     RTP        -236165.799162
      Recharge    330373.033372
Name: Volume, dtype: float64

### Customer Profile : 

In [58]:
def profile_customer(frame , id) : 
    user_df = frame[frame["ID_User"] == id]
    solde = round(user_df["amount_solde"].sum(),2)
    customer_monthly_ops = user_df.groupby([user_df.Date.dt.month, "Type"]).agg("Volume").count()
    process_rtp_rate = round(len(user_df[(user_df["Type"] == "RTP") & user_df["Status"] == 1]) / len(user_df[(user_df["Type"] == "RTP")])*100,2)
    return {
        "solde" : solde,
        "customer_monthly_ops" : customer_monthly_ops,
        "process_rtp_rate": process_rtp_rate
    }
    

In [51]:
profile_customer(df, 1)["solde"]

-573.99

In [53]:
profile_customer(df, 1)["process_rtp_rate"]

96.24

In [48]:
df["amount_solde"] = df["Volume"] /100

In [63]:
df.groupby(df.Date.dt.month).agg("Volume").count()

Date
1    226646
2    223879
3    245659
4    238903
5    113488
Name: Volume, dtype: int64